# 5. Feladatsor: megoldások

*(Nemlineáris egyenletek: visszatekintés a gradiens-ereszkedésre, Newton-iteráció)*

### P1. Feladat
Gondoljuk meg, hogy az $x_0 = 2^{255}$ pontból indulva az

a) $x_02^{-n}$

b) $x_02^{-n^2}$

c) $x_02^{-2^{n}}$

sorozatok hány lépés alatt érnek be a $0$ pont $\frac12$ sugarú, zárt gömbi környezetébe, majd írjunk programot és mérjük is meg, hogy valóban jól gondolkodtunk.

In [ ]:
from typing import Callable, Iterable
from functools import wraps

from itertools import dropwhile, accumulate, repeat, starmap

def first(x): return x[0]
def last(x): return x[-1]

def index_of_first(condition: Callable[(float,), bool], iterable: Iterable[float]):
    return first(next(
            dropwhile(
                lambda n_xn: not condition(last(n_xn)), 
                enumerate(iterable)
            )
    ))
    
def somuchfun(rule: Callable[(int, float), float], x0: float) -> int:
    return index_of_first(
        lambda x: x <= .5 + 1e-16, 
        starmap(rule, enumerate(repeat(x0)))
    )

In [ ]:
somuchfun(lambda n, x: x*2**(-n), 2**255)

In [ ]:
somuchfun(lambda n, x: x*2**(-n**2), 2**255)

In [ ]:
somuchfun(lambda n, x: x*2**(-2**n), 2**255)

### P2. Feladat

Alkalmazzuk a Newton-iterációt az $f(x, y) = \sin^2(x) + \sin^2(y) $ függvény $0$-hoz legközelebb eső minimumhelyének meghatározására, ha tudjuk, hogy az a nulla körül konvex, az alábbiak szerint:

a) Számoljuk ki $f$ deriváltját.
 
b) Az $(\frac14, \frac14)$ pontból indítsunk Newton-iterációt az $f'$ (egy) gyökének meghatározására.

In [ ]:
import numpy as np

from matplotlib import pyplot as plt

xs = np.linspace(-np.pi/2, np.pi/2, 301)
X, Y = np.meshgrid(xs, xs)

fig, ax = plt.subplots(
    figsize=(10,10),
    subplot_kw={"projection": "3d"}
)
ax.plot_surface(X, Y, np.sin(X)**2 + np.sin(Y)**2, cmap='jet')
ax.set_title(r"$\sin(x)^2 + \sin(y)^2$")

In [ ]:
from typing import Callable, Iterable, Any
from functools import wraps

from itertools import dropwhile, accumulate, repeat

# :(
A = Any

def iterate(f: Callable[(A,), A]) -> Callable[(A,), Iterable[A]]:
    @wraps(f)
    def _(x0):
        return accumulate(
          repeat(x0),
          lambda x,_: f(x)
        )
    return _

In [ ]:
from dataclasses import dataclass
from __future__ import annotations

V = np.ndarray
Matrix = np.matrix

FixedPointIterationState = tuple[int, V, V]

def initial_state(x: V) -> FixedPointIterationState:
    return 0, np.inf*x, x

@dataclass
class StoppingCondition:
    norm: Callable[(V,), float]
    atol: float
    rtol: float
    max_steps: int

    def holds(self, state: FixedPointIterationState) -> bool:
        n, d, x = state

        if n > self.max_steps:
            return False

        abs_err = self.norm(d)
        return all([
            abs_err <= self.atol,
            abs_err <= self.rtol * self.norm(x)
        ])

    def does_not_hold(self, state: FixedPointIterationState) -> bool:
        return not self.holds(state)


@dataclass
class IterationResult:
    x: V
    success: bool
    abs_err: float
    rel_err: float
    step_num: int

    @staticmethod
    def from_state_and_stopping_condition(
        state: FixedPointIterationState,
        stopping_condition: StoppingCondition
    ) -> IterationResult:
        n, d, x = state

        abs_err = stopping_condition.norm(d)
        return IterationResult(
            x=x,
            success=stopping_condition.holds(state),
            abs_err=abs_err,
            rel_err=abs_err / (1e-16 + stopping_condition.norm(x)),
            step_num=n,
        )

def fixpoint_iteration(
    displacement: Callable[(V,), V],
    stopping_condition: StoppingCondition,
    x0: V
) -> IterationResult:
    
    def step(state: FixedPointIterationState) -> FixedPointIterationState:
        n, d, x = state
        d1 = displacement(x)
        return n+1, d1, x + d1
    
    final_state = next(dropwhile(
        stopping_condition.does_not_hold,
        iterate(step)(initial_state(x0)),
    ))

    return IterationResult.from_state_and_stopping_condition(
        final_state,
        stopping_condition
    )

In [ ]:
def newton_displacement(
    f: Callable[(V,), V],
    df: Callable[(V,), Matrix],
) -> Callable[(V,), V]:
    def displacement(x: V) -> V:
        return np.linalg.solve(df(x), -f(x))
    return displacement
    
def newton_iteration(
    f: Callable[(V,), V],
    df: Callable[(V,), Matrix],
    stopping_condition: StoppingCondition,
    x0: V
) -> IterationResult:
    return fixpoint_iteration(newton_displacement(f, df), stopping_condition, x0)

In [ ]:
stopping_condition = StoppingCondition(
    np.linalg.norm,
    atol=1e-4,
    rtol=1e-4,
    max_steps=10**10,
)

In [ ]:
def f(x: V) -> V:
    return np.sin(2*x)

def df(x: V) -> Matrix:
    return np.diag(2*np.cos(2*x))

In [ ]:
newton_iteration(
    f, df, stopping_condition, .25*np.ones((2,))
)

### P3. Feladat

Tekinsük az alábbi egyenletrendszert:
$$
\begin{cases}
5 x^2 - y^2 &= 0,\\
y - 0.25(\sin x +\cos y) &= 0.
\end{cases}
$$

Newton-iteráció segítségével keressük ennek a $(\frac12, \frac12)$ pont környezetében lévő megoldását!

In [ ]:
def f(xy: V) -> V:
    x, y = xy
    return np.array([
        5*x**2 - y**2,
        y - .25*(np.sin(x) + np.cos(y))
    ])

def df(xy: V) -> Matrix:
    x, y = xy
    return np.matrix([
        [10*x, -2*y],
        [-.25*np.cos(x), 1 + .25*np.sin(y)],
    ])

In [ ]:
res = newton_iteration(
    f, df, stopping_condition, .5*np.ones((2,))
)
res

In [ ]:
f(res.x)